In [14]:
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from mistralai import Mistral
import dotenv
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

FOLDER_PARENTS = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
index = faiss.read_index(os.path.join(FOLDER_PARENTS, "my_rag_db.index"))

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1776.50it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [16]:
with open(os.path.join(FOLDER_PARENTS, "my_rag_db.json"), "r") as f:
    metadata = json.load(f)

In [20]:
def retrieve_context(question, k=3):
    '''Look for the k passages most useful'''
    query_vector = embedder.encode([question], convert_to_numpy=True)
    
    distances, indices = index.search(query_vector, k)
    
    results = []
    
    for i in range(k):
        idx = indices[0][i]
        if idx == -1:
            continue
        
        doc = metadata[idx]
        
        results.append({
            "text": doc["text"],
            "url": doc["url"],
            "score": float(distances[0][i])
        })
        
    return results

In [23]:
def generate_answer(question):
    context_results = retrieve_context(question, k=3)
    
    if not context_results:
        return "I did not find relevant information in my documents."
    
    context_str = ""
    for i, res in enumerate(context_results):
        context_str += f"Source [{i+1}] ({res['url']}):\n{res['text']}\n\n"
    
    system_prompt = """
    Tu es un assistant factuel et précis. Ta mission est de répondre à la question de l'utilisateur 
    en utilisant UNIQUEMENT les informations fournies dans le contexte ci-dessous.
    
    Règles strictes :
    - Si la réponse n'est pas dans le contexte, dis "Je ne sais pas".
    - Ne jamais inventer d'information.
    - Cite tes sources à la fin de chaque phrase ou paragraphe en utilisant le format [1], [2].
    - Réponds dans la même langue que la question (Français).
    """
    
    user_message = f"""
    CONTEXTE: {context_str}
    
    QUESTION: {question}
    """
    
    chat_response = client.chat.complete(
        model="mistral-small-latest",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    
    return chat_response.choices[0].message.content


In [24]:
if __name__ == "__main__":
    print("Bienvenue dans ton RAG ! (Tape 'q' pour quitter)")
    
    while True:
        user_input = input("\nTa question : ")
        if user_input.lower() in ['q']:
            break
            
        response = generate_answer(user_input)
        
        print("\n--- RÉPONSE MISTRAL ---")
        print(response)
        print("-----------------------")

Bienvenue dans ton RAG ! (Tape 'q' pour quitter)

--- RÉPONSE MISTRAL ---
Je ne sais pas.
-----------------------


In [25]:
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)
embedder = SentenceTransformer("all-MiniLM-L6-v2")

FOLDER_PARENTS = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
index = faiss.read_index(os.path.join(FOLDER_PARENTS, "my_rag_db.index"))

with open(os.path.join(FOLDER_PARENTS, "my_rag_db.json"), "r") as f:
    metadata = json.load(f)

def retrieve_context(question, k=3):
    """Look for the k passages most useful"""
    query_vector = embedder.encode([question], convert_to_numpy=True)
    distances, indices = index.search(query_vector, k)
    results = []
    for i in range(k):
        idx = indices[0][i]
        if idx == -1:
            continue
        doc = metadata[idx]
        results.append({
            "text": doc["text"],
            "url": doc["url"],
            "score": float(distances[0][i])
        })
    return results

def generate_answer(question):
    context_results = retrieve_context(question, k=3)
    if not context_results:
        return "I did not find relevant information in my documents."

    context_str = ""
    for i, res in enumerate(context_results):
        context_str += f"Source [{i+1}] ({res['url']}):\n{res['text']}\n\n"

    system_prompt = """
    Tu es un assistant factuel et précis. Ta mission est de répondre à la question de l'utilisateur 
    en utilisant UNIQUEMENT les informations fournies dans le contexte ci-dessous.

    Règles strictes :
    - Si la réponse n'est pas dans le contexte, dis "Je ne sais pas".
    - Ne jamais inventer d'information.
    - Cite tes sources à la fin de chaque phrase ou paragraphe en utilisant le format [1], [2].
    - Réponds dans la même langue que la question (Français).
    """

    user_message = f"""
    CONTEXTE: {context_str}

    QUESTION: {question}
    """

    chat_response = client.chat.complete(
        model="mistral-small-latest",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return chat_response.choices[0].message.content

if __name__ == "__main__":
    print("Bienvenue dans ton RAG ! (Tape 'q' pour quitter)")
    while True:
        user_input = input("\nTa question : ")
        if user_input.lower() in ["q"]:
            break
        response = generate_answer(user_input)
        print("\n--- RÉPONSE MISTRAL ---")
        print(response)
        print("-----------------------")

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1479.02it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Bienvenue dans ton RAG ! (Tape 'q' pour quitter)

--- RÉPONSE MISTRAL ---
The Walking Dead season 8 premiered on October 22, 2017 [1].
-----------------------

--- RÉPONSE MISTRAL ---
Je ne sais pas.
-----------------------

--- RÉPONSE MISTRAL ---
Je ne sais pas.
-----------------------


KeyboardInterrupt: 